In [1]:
import argparse
import importlib
import json
import logging
import os
import time
import sys
import numpy as np

from matplotlib import pyplot as plt
from tensorforce import TensorForceError
from tensorforce.agents import DQNAgent
from tensorforce.execution import Runner
from surface_env import *
import ase.io


/home/zulissi/miniconda3/envs/tensorforce/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/zulissi/miniconda3/envs/tensorforce/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/zulissi/miniconda3/envs/tensorforce/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/zulissi/miniconda3/

In [2]:
def plot_energy(energy, ylabel, save_path):
    plt.figure()
    plt.xlabel('training episode')
    plt.ylabel(ylabel)
    plt.title('episode vs. ' + ylabel)
    plt.plot(energy)
    plt.savefig(save_path)
    print('figure saved as {}'.format(save_path))
    return 

In [3]:
episodes=100000
horizon=100
deterministic=False

In [ ]:

env = SurfaceEnv(horizon)
print('Initial energy:', env.get_energy())

# lattice = Surface()
# lattice.reset()
# print(lattice.current_positions())
# print(lattice.current_positions().shape)
# print(type(lattice.current_positions()))

network_spec = [
    {
        "type": "dense",
        "size": 64,
        "activation": "relu"
    },
    {
        "type": "dense",
        "size": 32,
        "activation": "relu"
    }
    # {
    #     "type": "dense",
    #     "size": 6+32,
    #     "activation": "softmax"
    # }
]

# print(env.states)
# print(env.states['shape'])

agent = DQNAgent(
    states=env.states,
    actions=env.actions,
    network=network_spec,
    batched_observe=True, 
    batching_capacity=8000,
    execution=dict(
        type='single',
        session_config=None,
        distributed_spec=None
    ), 

    states_preprocessing=None,
    reward_preprocessing=None,

    update_mode=dict(
        unit='timesteps',
        batch_size=10,
        frequency=10
    ),
    memory=dict(
        type='replay',
        include_next_states=True,
        capacity=40000
    ),

    optimizer=dict(
        type='clipped_step',
        clipping_value=0.1,
        optimizer=dict(
            type='adam',
            learning_rate=1e-3
        )
    ),
    actions_exploration=dict(
        type='epsilon_anneal',
        initial_epsilon=0.5,
        final_epsilon=0.05,
        timesteps=1000000
    ),
    discount=1,
    distributions=None,
    entropy_regularization=0.01,
    target_sync_frequency=1000,
    target_update_weight=1.0,
    double_q_model=False,
    huber_loss=None,

    summarizer=dict(
        directory=None,
        labels=['graph', 'total-loss']
    ),
)

runner = Runner(
    agent=agent,
    environment=env,
    repeat_actions=1
)

cur_min = -1e6

def episode_finished(r):
    
    global cur_min
    # positions = env.get_positions()
    # print("pos diff:", np.sum(positions[1,:] - positions[0,:]))
    # np.save('shit', positions)
    
    if r.episode % 100 == 0 or r.episode_rewards[-1]>cur_min:
        final_energy = env.get_energy()
        positions = env.get_positions()
        
        image_list = []
        with ase.io.trajectory.TrajectoryWriter('gifs/%d_%1.2f_%1.2f.traj'%(r.episode,final_energy, r.episode_rewards[-1])) as traj:
            for pos in positions:
                env._lattice.set_free_atoms(pos)
                copy_atoms = env._lattice.atoms.copy()
                image_list.append(copy_atoms.repeat((2,2,1)))
                traj.write(copy_atoms, energy=env.get_energy())

#         ase.io.write('gifs/%d_%1.2f.gif'%(r.episode,final_energy) , image_list, interval=100)
        
        if final_energy > cur_min:
            cur_min = r.episode_rewards[-1]
    if r.episode % 50 == 0:
        positions = env.get_positions()
        pos_fn = '_'.join(['pos_3', str(r.episode)])
        pos_dir = os.path.join('new_pos', pos_fn)
        np.save(pos_dir, positions)
    if r.episode % 50 == 0:
        agent_fn = '_'.join(['agent_3', str(r.episode)])
        agent_dir = os.path.join('new_agents', agent_fn)
        r.agent.save_model(agent_dir)
        print("Saving agent to {}".format(agent_dir))
    if r.episode % 50 == 0:
        rew_fn = '.'.join(['_'.join(['reward_3', str(r.episode)]), 'png'])
        rew_dir = os.path.join('new_plots', rew_fn)
        plot_energy(r.episode_rewards, 'accumulated reward', rew_dir)
        energy_fn = '.'.join(['_'.join(['final', 'energy_3', str(r.episode)]), 'png'])
        energy_dir = os.path.join('new_plots', energy_fn)
        plot_energy(env.final_energy, 'final energy', energy_dir)
    print("Finished episode {ep} after {ts} timesteps (reward: {reward})".
        format(ep=r.episode, ts=r.episode_timestep,reward=r.episode_rewards[-1]))
    return True

runner.run(
    # num_timesteps=args.timesteps,
    num_episodes=episodes,
    max_episode_timesteps=horizon,
    deterministic=deterministic,
    episode_finished=episode_finished
)
runner.close()

W0813 20:09:04.796082 139962960201536 deprecation_wrapper.py:119] From /home/zulissi/miniconda3/envs/tensorforce/lib/python3.7/site-packages/tensorforce/models/model.py:252: The name tf.variable_scope is deprecated. Please use tf.compat.v1.variable_scope instead.

W0813 20:09:04.796555 139962960201536 deprecation_wrapper.py:119] From /home/zulissi/miniconda3/envs/tensorforce/lib/python3.7/site-packages/tensorforce/models/model.py:457: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0813 20:09:04.797903 139962960201536 deprecation_wrapper.py:119] From /home/zulissi/miniconda3/envs/tensorforce/lib/python3.7/site-packages/tensorforce/core/explorations/exploration.py:42: The name tf.make_template is deprecated. Please use tf.compat.v1.make_template instead.

W0813 20:09:04.818248 139962960201536 deprecation_wrapper.py:119] From /home/zulissi/miniconda3/envs/tensorforce/lib/python3.7/site-packages/tensorforce/models/model.py:846: The name tf.get_variabl

Initial energy: 2.1797225799832667


  0%|          | 0/100000 [00:00<?, ?it/s]

Episode ends and final energy: 2.203453130743803


  0%|          | 1/100000 [00:04<130:43:19,  4.71s/it]

...

Episode ends and final energy: 3.9310592562964874
Finished episode 43 after 99 timesteps (reward: -1950.532636394462)


  0%|          | 44/100000 [03:25<129:53:31,  4.68s/it]

Episode ends and final energy: 2.182536380702908
Finished episode 44 after 99 timesteps (reward: -176.90678013080816)


  0%|          | 45/100000 [03:30<129:01:27,  4.65s/it]

Episode ends and final energy: 5.734890622595401
Finished episode 45 after 99 timesteps (reward: -5426.771972529091)
